In [1]:
import openai
import pandas as pd
df = pd.read_csv('olympics_qa.csv')
df.head()

,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,The 2020 Summer Olympics (Japanese: 2020年夏季オリン...,716,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,1. What is the main idea of the passage?\n2. W...,1. The main idea of the passage is that the 20...
1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126,2020 Summer Olympics\nHost city selection\n\nT...,1. Which city was selected as the host city of...,1. Tokyo\n2. Istanbul\n3. Tokyo\n4. Istanbul\n...
2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",375,2020 Summer Olympics\nImpact of the COVID-19 p...,1. What is the impact of the COVID-19 pandemic...,1. The impact of the COVID-19 pandemic on the ...
3,2020 Summer Olympics,Qualifying event cancellation and postponement,Concerns about the pandemic began to affect qu...,298,2020 Summer Olympics\nQualifying event cancell...,1. What is the name of the pandemic that affec...,1. The name of the pandemic was COVID-19.\n2. ...
4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163,2020 Summer Olympics\nEffect on doping tests\n...,1. What is the main reason for the reduced tes...,1. The main reason for the reduced testing is ...


In [10]:
df['questions_new'] = df['questions'].str.split('\n')
df_no_duplicates = df.drop_duplicates(subset='questions_new', keep='first')
df_no_duplicates.drop('questions_new', axis=1, inplace=True)
df_no_duplicates.head()

/var/folders/8v/dvjm79453xx0ft_dn74t6hvm0000gy/T/ipykernel_79418/1889464017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates.drop('questions_new', axis=1, inplace=True)


,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,The 2020 Summer Olympics (Japanese: 2020年夏季オリン...,716,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,1. What is the main idea of the passage?\n2. W...,1. The main idea of the passage is that the 20...
1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126,2020 Summer Olympics\nHost city selection\n\nT...,1. Which city was selected as the host city of...,1. Tokyo\n2. Istanbul\n3. Tokyo\n4. Istanbul\n...
2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",375,2020 Summer Olympics\nImpact of the COVID-19 p...,1. What is the impact of the COVID-19 pandemic...,1. The impact of the COVID-19 pandemic on the ...
3,2020 Summer Olympics,Qualifying event cancellation and postponement,Concerns about the pandemic began to affect qu...,298,2020 Summer Olympics\nQualifying event cancell...,1. What is the name of the pandemic that affec...,1. The name of the pandemic was COVID-19.\n2. ...
4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163,2020 Summer Olympics\nEffect on doping tests\n...,1. What is the main reason for the reduced tes...,1. The main reason for the reduced testing is ...


In [11]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(640, 160)

In [12]:
df.context.str.contains('->').sum()

0

In [15]:
df = df[df.tokens<2000]

def create_fine_tuning_dataset(df, discriminator=False, n_negative=1, add_related=False):
    """
    Create a dataset for fine tuning the OpenAI model; either for a discriminator model, 
    or a model specializing in Q&A, where it says if no relevant context is found.

    Parameters
    ----------
    df: pd.DataFrame
        The dataframe containing the question, answer and context pairs
    discriminator: bool
        Whether to create a dataset for the discriminator
    n_negative: int
        The number of random negative samples to add (using a random context)
    add_related: bool
        Whether to add the related contexts to the correct context. These are hard negative examples

    Returns
    -------
    pd.DataFrame
        The dataframe containing the prompts and completions, ready for fine-tuning
    """
    rows = []
    for i, row in df.iterrows():
        for q, a in zip(("1." + row.questions).split('\n'), ("1." + row.answers).split('\n')):
            if len(q) >10 and len(a) >10:
                if discriminator:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" yes"})
                else:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" {a[2:].strip()}"})

    for i, row in df.iterrows():
        for q in ("1." + row.questions).split('\n'):
            if len(q) >10:
                for j in range(n_negative + (2 if add_related else 0)):
                    random_context = ""
                    if j == 0 and add_related:
                        # add the related contexts based on originating from the same wikipedia page
                        subset = df[(df.title == row.title) & (df.context != row.context)]
                        
                        if len(subset) < 1:
                            continue
                        random_context = subset.sample(1).iloc[0].context
                    if j == 1 and add_related:
                        # add the related contexts based on the most similar contexts according to the search
                        random_context = get_random_similar_contexts(q[2:].strip(), row.context, search_model='ada', max_rerank=10)
                    else:
                        while True:
                            # add random context, which isn't the correct context
                            random_context = df.sample(1).iloc[0].context
                            if random_context != row.context:
                                break
                    if discriminator:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" no"})
                    else:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" No appropriate context found to answer the question."})

    return pd.DataFrame(rows) 

In [16]:
for name, is_disc in [('discriminator', True), ('qa', False)]:
    for train_test, dt in [('train', train_df), ('test', test_df)]:
        ft = create_fine_tuning_dataset(dt, discriminator=is_disc, n_negative=1, add_related=True)
        ft.to_json(f'{name}_{train_test}.jsonl', orient='records', lines=True)

NameError: name 'get_random_similar_contexts' is not defined